In [3]:
import pandas as pd
import numpy as np
import anndata as ad
import scanpy as sc
import scipy.sparse as sp
import glob
from tqdm import tqdm
import matplotlib.pyplot as plt

In [4]:
def random_subset(adata,cell_num=1200):
    random_indices = np.random.choice(adata.n_obs, cell_num, replace=True)   # 从 276593 个细胞中随机选取 3000 个细胞的索引
    adata_subset = adata[random_indices, :]   # 使用选取的索引创建新的 AnnData 对象

    return adata_subset

In [6]:
dataset_dir = '/home/hanyuji/Data/ST_data/mouse_hippocampus/cell/mouse1/'

cell_macaque1 = ['T305.h5ad', 'T308.h5ad', 'T312.h5ad', 'T315.h5ad', 'T318.h5ad', 'T322.h5ad', 'T325.h5ad', 'T330.h5ad', 'T333.h5ad', 'T336.h5ad', 'T339.h5ad',
 'T306.h5ad', 'T309.h5ad', 'T313.h5ad', 'T316.h5ad', 'T319.h5ad', 'T323.h5ad', 'T326.h5ad', 'T331.h5ad', 'T334.h5ad', 'T337.h5ad', 'T340.h5ad', 'T307.h5ad',
 'T311.h5ad', 'T314.h5ad', 'T317.h5ad', 'T321.h5ad', 'T324.h5ad', 'T329.h5ad', 'T332.h5ad', 'T335.h5ad', 'T338.h5ad', 'T341.h5ad']

adatas = []
for file_name in tqdm(cell_macaque1):
    adata = sc.read_h5ad(dataset_dir+file_name)
    sc.pp.filter_genes(adata, min_counts = 5)
    subset = random_subset(adata,1000)
    adatas.append(subset)

combined_adata = ad.concat(
    adatas, 
    axis=0, 
    join='outer', 
    merge='unique', 
    uns_merge='unique', 
    label='batch', 
    keys=list(range(1, len(adatas)+1)), 
    index_unique='-', 
    fill_value=0
)


100%|██████████| 33/33 [00:03<00:00,  8.42it/s]
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [7]:
combined_adata

AnnData object with n_obs × n_vars = 33000 × 18960
    obs: 'region', 'cellID', 'x', 'y', 'cellType', 'batch'
    obsm: 'spatial'

In [8]:
sc.pp.highly_variable_genes(combined_adata, n_top_genes=2000 ,flavor='seurat_v3')
combined_adata = combined_adata[:, combined_adata.var.highly_variable]
# 标准化 log 
sc.pp.normalize_total(combined_adata, target_sum=1e4)
sc.pp.log1p(combined_adata)

/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:206: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/anndata/_core/anndata.py:1818: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/hanyuji/miniconda3/envs/PASTE/lib/python3.9/site-packages/scanpy/preprocessing/_normalization.py:233: UserWarning: Some cells have zero counts
  warn(UserWarning("Some cells have zero counts"))


In [9]:
combined_adata.write('/home/hanyuji/Workbench/ST/paste_alignment_cortex/PASTE_align/mouse_hippocampus_cell1_33slices_1000spot_2000gene.h5ad')